In [1]:
library(tidyverse)
library(rvest)

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.1     ✔ purrr   1.0.1
✔ tibble  3.1.8     ✔ dplyr   1.1.0
✔ tidyr   1.3.0     ✔ stringr 1.4.1
✔ readr   2.1.4     ✔ forcats 1.0.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘rvest’


The following object is masked from ‘package:readr’:

    guess_encoding




# Lecture 11: Web scraping

<div style="border: 1px double black; padding: 10px; margin: 10px">

**After today's lecture you will:**
* Understand how to import data from online sources by scraping web pages.
</div>

These notes correspond to Chapter 26 of your book.


## Ethics of scraping data online
You should carefully read [Section 26.2](https://r4ds.hadley.nz/webscraping.html#scraping-ethics-and-legalities) of the book concerning various ethical and legal issues surrounding scraping web sites for data. In this class we will only look at large, public web sites like Wikipedia and IMDB, where there is no risk of anything bad happening. However, there are other situations where it may be unethical, or even illegal, to harvest data from a website, even if you are technically able. **As data scientists in the real world, it will be up to you to carefully weigh these concerns before using the tools discussed in today's lecture.**

## Reading data from the Internet
These days, it's increasingly common to pull data from online sources. For example, say I wanted to know the population of European countries. This is [easily found](https://en.wikipedia.org/wiki/Demographics_of_Europe#Population_by_country) on Wikipedia. How can I get these data into R and analyze them?

## How do web pages work?

Web pages are written in a special language called HTML (**H**yper**t**ext **M**arkup **L**anguage). Here is a simple example of some HTML:

    <html>
    <head> 
      <title>Page title</title>
    </head>
    <body>
      <h1 id='first'>A heading</h1>
      <p>Some text &amp; <b>some bold text.</b></p>
      <img src='myimg.png' width='100' height='100'>
    </body>
    </html>

Web scraping is possible because most web pages have a consistent, hierarchical structure. For example, if I asked you how to navigate to the title of the web page shown above, you would follow the "path"

    html > head > title
    
to arrive at "Page title".

## HTML elements

There are a lot of HTML elements that might contain interesting information. Here are a few of the most common:
- Block tags that denote sections of text: `<h1>` (heading), `<p>` (paragraph), `<ul>`/`<ol>` (un)ordered list, etc.
- `<table>` (a table), `<tr>` (a table row), `<td>` (a table cell), etc.
- Each of these elements can contain attributes such as `id=` or `class=`. For example, `<table id="movies">` is probably a table that contains movie information.

The `rvest` package is used to load a web page and extract elements and tables based on their HTML tags. Let's see how it works by scraping the Wikipedia page mentioned earlier:

In [3]:
library(rvest)
europop <- read_html("http://en.wikipedia.org/wiki/Demographics_of_Europe#Population_by_country")
europop

{html_document}
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-language-alert-in-sidebar-enabled vector-feature-sticky-header-disabled vector-feature-page-tools-disabled vector-feature-page-tools-pinned-disabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled" lang="en" dir="ltr">
[1] <head>\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8 ...
[2] <body class="skin-vector skin-vector-search-vue vector-toc-pinned mediawi ...

In this page there are many tables:

In [ ]:
wiki_tables <- europop %>% 
  html_elements("table.wikitable.sortable") %>% 
    html_table
wiki_tables 

We will get the third table, select the first two columns, slice off the first row using 'slice()' function and then convert the datatypes of the columns using
* as.integer
* parse_number

In [13]:
#
wiki_tables[[3]] %>% select(1:3) %>% slice(-1) %>%
  mutate(
  Year = as.integer(Year),
  Averagepopulation = parse_number(Averagepopulation),
  b = parse_number(`Live births`))

Year,Averagepopulation,Live births,b
<int>,<dbl>,<chr>,<dbl>
1950,549721718,"12,202,220",12202220
1951,554559502,"12,112,425",12112425
1952,559609904,"12,142,368",12142368
1953,565058633,"12,120,826",12120826
1954,570670994,"12,151,779",12151779
1955,576304974,"12,134,270",12134270
1956,581975516,"12,133,583",12133583
1957,587711635,"12,194,100",12194100
1958,593669297,"12,177,600",12177600


How can we find the correct one? One option is to use our browser to find something that uniquely identifies the table that we want. Alternatively, if a small number, we can just use index number to find the one we want:

In [ ]:
# find the table that contains the population for each country


## 🤔 Quiz

What's the average population density ($\text{persons}/\text{km}^2$) for countries in Europe?

<ol style="list-style-type: upper-alpha;">
    <li>1234.5</li>
    <li>20000.0</li>
    <li>611.8</li>
    <li>6520.5</li>
    <li>101.1</li>
</ol>



In [20]:
wiki_tables <- europop %>% html_elements("table.wikitable.sortable") %>% html_table
wiki_tables[[4]] %>% colnames

[1] "Country (or territory)"                                    
[2] "Population[1][2]"                                          
[3] "Area.mw-parser-output .nobold{font-weight:normal}(km2)[14]"
[4] "Density(per km2)"                                          
[5] "Capital"

In [18]:
wiki_tables[[4]] %>% 
  select(dens = `Density(per km2)`) %>% 
  mutate(dens = parse_number(dens)) %>%
    summarise(avg = mean(dens))

avg
<dbl>
611.819


## 🤔 Quiz

Use the same page Wikipedia page (Demographics of Europe) to answer the following question:

On average, how many people were born *each day* in Europe between 2010 and 2021 (inclusive)?

<ol style="list-style-type: upper-alpha;">
    <li>90210.10</li>
    <li>23043.97</li>
    <li>7710127</li>
    <li>21123.64</li>
    <li>21109.18</li>
</ol>



In [29]:
library(lubridate)
# lubridate::make_date
difftime(make_date(2022, 1, 1), make_date(2010, 1, 1), "days")

# 366 + 365


Time difference of 4383 days

In [31]:
wiki_tables[[3]] %>% select(1:3) %>% slice(-1) %>%
  mutate(Year = as.integer(Year),
Averagepopulation = parse_number(Averagepopulation),
b = parse_number(`Live births`)) %>% filter(Year >= 2010) %>%
  summarise(avg_birth = sum(b)/4383)

avg_birth
<dbl>
21109.18


### The UofM Stats department
Let's say I wanted to make a table of all the [undergraduate stats courses](https://lsa.umich.edu/stats/undergraduate-students/statistics-courses.html) offered by the department. 

In [37]:
stats <- read_html('https://lsa.umich.edu/stats/undergraduate-students/statistics-courses.html')
stats %>% html_elements("b") %>% html_text

[1] "STATS 100: Introduction to Statistics, Probability and Mathematical Modeling"
 [2] "STATS 125: Statistics, Games and Coincidences"                               
 [3] "STATS 150: Making Sense of Data"                                             
 [4] "STATS 180: AP Statistics"                                                    
 [5] "STATS 206/DATASCI 101: Introduction to Data Science"                         
 [6] "STATS 250: Introduction to Statistics and Data Analysis"                     
 [7] "STATS 251 – Introductory Statistics Supplement for Mathematics Education"    
 [8] "STATS 280: Honors Introduction to Statistics & Data Analysis"                
 [9] "STATS 299 - Workplace Internship for Undergraduate Statistics Majors"        
[10] "STATS/DATASCI 306: Introduction to Statistical Computing"                    
[11] "STATS/DATASCI 315 – Statistics and Artificial Intelligence "                 
[12] ""                                                                            
[13] "STATS 401: Applied Statistical Methods II"                                   
[14] "STATS 403: Introduction to Quantitative Research Methods"                    
[15] "STATS 404: Effective Communication in Statistics"                            
[16] "STATS/DATASCI 406: Computational Methods in Statistics and Data Science"     
[17] "STATS 412: Introduction to Probability & Statistics"                         
[18] "STATS/DATASCI 413: Applied Regression Analysis"                              
[19] "STATS 414: Topics Course"                                                    
[20] "STATS/DATASCI 415: Data Mining and Statistical Learning"                     
[21] "STATS/MATH 425: Introduction to Probability"                                 
[22] "STATS 426: Introduction to Theoretical Statistics"                           
[23] "STATS 430: Applied Probability"                                              
[24] "STATS/BIOSTAT 449: Topics in Biostatistics"                                  
[25] "STATS/DATASCI 451: Bayesian Data Analysis"                                   
[26] "STATS 470: Introduction to the Design of Experiments"                        
[27] "STATS 480: Survey Sampling Techniques"                                       
[28] "STATS/DATASCI 485: Capstone Seminar"                                         
[29] "STATS 489: Independent Study in Statistics"                                  
[30] "STATS 499: Honors Seminar"                                                   
[31] "Please send permission requests to statsugradprogram@umich.edu"

How should we extract the data from this web page? We notice from inspecting the page that each course title is a `<b>` (bold) element:

## IMDB top movies

Let's consider a well-known table: the [top 250 movies on IMDB](https://www.imdb.com/chart/top/).

In [ ]:
imdb.250 <- read_html("https://www.imdb.com/chart/top/")

In [ ]:
# parse imdb
imdbf %>% arrange(year) %>% mutate(top_rating = cummax(rating), king = rating == top_rating) %>% 
    filter(king) %>% mutate(delta = lead(year) - year)


title,year,rating,top_rating,king,delta
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>
The Kid,1921,8.2,8.2,TRUE,6
Metropolis,1927,8.2,8.2,TRUE,4
City Lights,1931,8.4,8.4,TRUE,5
Modern Times,1936,8.4,8.4,TRUE,4
The Great Dictator,1940,8.4,8.4,TRUE,2
Casablanca,1942,8.5,8.5,TRUE,4
It's a Wonderful Life,1946,8.6,8.6,TRUE,8
Seven Samurai,1954,8.6,8.6,TRUE,3
12 Angry Men,1957,9.0,9.0,TRUE,15


## Exercise

"The Kid" came out in 1921 and has a rating of 8.2. Another movie that was rated at least as high didn't come out until 1927 (Metropolis), so we could say that The Kid reigned as the #1 film for six years. Metropolis reigned for four years until City Lights (rating 8.4) came out.

Which film reigned for the longest amount of time?

In [ ]:
# longest reign

## Super Bowl TV ratings
We just had the Super Bowl. How have the TV ratings for the Super Bowl changed over the years?

In [ ]:
sbtv <- read_html('https://en.wikipedia.org/wiki/Super_Bowl_television_ratings') %>% html_elements('table') %>% .[[1]] %>% html_table

In [ ]:
# viewers over time

How does this compare with other major sports?

- https://en.wikipedia.org/wiki/World_Series_television_ratings
- https://en.wikipedia.org/wiki/NBA_Finals_television_ratings

In [ ]:
# super bowl vs world series

### Reddit
Let's see how to scrape the [UofM Reddit site](https://old.reddit.com/r/uofm):

In [ ]:
top.reddit <- read_html('https://old.reddit.com/r/uofm/top/?sort=top&t=all')

Let's plot the top scoring posts, when they were posted, and how many votes they have received.

In [ ]:
# top posts on r/uofm